# Q1: Implementing and Training an MLP

In [ ]:
import csv

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.metrics import accuracy_score

from models.mlp_model import MLP
from training_testing.training import perform_hyperparameter_search, train
from utilities import load_data

## Additional Utilities

## Two Gaussians

In [ ]:
X_train, y_train = load_data("data/two_gaussians_train.csv")
X_valid, y_valid = load_data("data/two_gaussians_valid.csv")
X_test, y_test = load_data("data/two_gaussians_test.csv")
y_train = y_train.reshape(-1, 1)
y_valid = y_valid.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)

csv_filename = "results/two_gaussians/hyperparameter_results.csv"

### Hyperparameter Search

In [ ]:
hidden_layer_sizes = [5, 10, 15, 20, 25, 30]
batch_sizes = [16, 32, 64]
learning_rates = [0.001, 0.01, 0.1]
epoch_values = [100, 150, 400]

In [ ]:
with open(csv_filename, mode="w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(
        [
            "Hidden Layers",
            "Batch Size",
            "Learning Rate",
            "Epochs",
            "Last Train Loss",
            "Last Validation Loss",
            "Last Train Accuracy",
            "Last Validation Accuracy",
        ]
    )

In [ ]:
perform_hyperparameter_search(
    hidden_layer_sizes,
    batch_sizes,
    learning_rates,
    epoch_values,
    X_train,
    y_train,
    X_valid,
    y_valid,
    csv_filename,
    "two_gaussians",
)

### Testing

In [ ]:
gaussian_results_df = pd.read_csv("results/two_gaussians/hyperparameter_results.csv")

In [ ]:
gaussian_results_df.sort_values(by="Last Validation Loss", ascending=True).head(10)

In [ ]:
lr = 0.1
batch_size = 32
k = 20
epochs = 200

In [ ]:
model = MLP(input_size=X_train.shape[1], hidden_size=k)

In [ ]:
train_losses, train_accuracies, val_losses, val_accuracies = train(
    model,
    X_train,
    y_train,
    X_valid,
    y_valid,
    lr=lr,
    epochs=epochs,
    batch_size=batch_size,
)

In [ ]:
test_pred = model.predict(X_test)
test_accuracy = accuracy_score(y_test.squeeze(), test_pred)

print(f"Test accuracy for k={k}, Batch={batch_size}, LR={lr}: {test_accuracy}")

### Visualizing Decision Boundary

In [ ]:
x_min, x_max = X_test[:, 0].min() - 1, X_test[:, 0].max() + 1
y_min, y_max = X_test[:, 1].min() - 1, X_test[:, 1].max() + 1
xx, yy = np.meshgrid(np.linspace(x_min, x_max, 100), np.linspace(y_min, y_max, 100))

grid_points = np.c_[xx.ravel(), yy.ravel()]
grid_predictions = model.predict(grid_points)
grid_predictions = grid_predictions.reshape(xx.shape)

In [ ]:
df_test = pd.DataFrame(X_test, columns=["Feature 1", "Feature 2"])
df_test["Label"] = y_test.squeeze()
df_test["Label"] = df_test["Label"].astype(int)

incorrect_predictions = test_pred != y_test.squeeze()

In [ ]:
plt.figure(figsize=(9, 5), dpi=120)
plt.contourf(xx, yy, grid_predictions, alpha=0.2, cmap="RdBu_r")

sns.scatterplot(
    data=df_test, x="Feature 1", y="Feature 2", hue="Label", palette=["blue", "red"]
)

incorrect_points = df_test[incorrect_predictions]
plt.scatter(
    incorrect_points["Feature 1"],
    incorrect_points["Feature 2"],
    facecolors="none",
    edgecolors="black",
    s=100,
    label="Incorrectly Classified",
)


plt.title("Gaussian Data with Decision Boundary (Test Set)")
plt.legend(title="Legend", loc="upper right")
plt.xlim(xx.min(), xx.max())
plt.ylim(yy.min(), yy.max())
plt.show()